# Langchain

In [ ]:
RESUME_AGENT_PROMPT = (
    "You are a Resume Parsing and Analysis Expert.\n"
    "Your primary responsibility is to extract text from resumes and answer user questions strictly based on the resume content.\n\n"

    "Rules:\n"
    "- ALWAYS use the `resume_parser` tool when a resume file path is provided.\n"
    "- Use ONLY the information found in the parsed resume text.\n"
    "- If the requested information is not present in the resume, clearly say that it is not available.\n"
    "- Do NOT make assumptions or add external knowledge.\n\n"

    "After parsing the resume:\n"
    "- Summarize relevant sections if needed (skills, experience, education, projects).\n"
    "- Answer the user’s question clearly and concisely using resume evidence.\n"
)


GENERAL_PROMPT = (
    "You are a General Question Answering Assistant.\n"
    "Your role is to answer general knowledge, conceptual, or conversational questions.\n\n"

    "Rules:\n"
    "- Use the `general_question_answer` tool to generate responses.\n"
    "- Provide clear, accurate, and easy-to-understand answers.\n"
    "- Keep responses concise unless the user asks for detailed explanations.\n"
    "- Do NOT perform web searches or resume parsing.\n\n"

    "Tone:\n"
    "- Friendly, professional, and helpful.\n"
)


WEB_SEARCH_PROMPT = (
    "You are a Web Search and Information Retrieval Expert.\n"
    "Your role is to search the internet for up-to-date and factual information.\n\n"

    "Rules:\n"
    "- ALWAYS use the `google_search` tool when the user asks for current, real-world, or factual information.\n"
    "- Do NOT answer from prior knowledge without searching.\n"
    "- Prefer recent and reliable sources.\n\n"

    "After retrieving results:\n"
    "- Summarize the most relevant findings clearly.\n"
    "- If multiple sources agree, present a consolidated answer.\n"
    "- If information is uncertain or conflicting, mention it.\n"
)


In [28]:
import os
from dotenv import load_dotenv  
from langchain_deepseek import ChatDeepSeek


load_dotenv()

DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# Initialize Azure OpenAI model
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=DEEPSEEK_API_KEY
    # api_key="...",
    # other params...
)

In [50]:
from langchain.tools import tool
import os
from dotenv import load_dotenv  
from langchain.agents import create_agent
from PyPDF2 import PdfReader
from docx import Document
from tavily import TavilyClient
from langgraph.checkpoint.memory import InMemorySaver
from langchain_deepseek import ChatDeepSeek
from langchain_apify import ApifyActorsTool



load_dotenv()

DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Initialize Azure OpenAI model
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=DEEPSEEK_API_KEY
    # api_key="...",
    # other params...
)

def extract_text_from_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def extract_text_from_docx(docx_path: str) -> str:
    doc = Document(docx_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])



@tool(description="Extracts text from PDF or DOCX resume files.")
def resume_parser(resume_file_path: str):
    if resume_file_path.endswith(".pdf"):
        return extract_text_from_pdf(resume_file_path)
    elif resume_file_path.endswith(".docx"):
        return extract_text_from_docx(resume_file_path)
    else:
        raise ValueError("Unsupported file type")

# Create resume parser agent
resume_parser_agent = create_agent(
    model,
    tools=[resume_parser],
    name="resume_parser_agent",
    system_prompt=RESUME_AGENT_PROMPT
)

@tool(description="Answer general questions")
def general_question_answer(question: str):
    response = model.invoke(question)
    return response.content

# Create general Q&A agent
general_question_answer_agent = create_agent(
    model,
    tools=[general_question_answer],
    name="general_question_answer_agent",
    system_prompt=GENERAL_PROMPT
)

@tool(description="Do a web search based on user queries")
def google_search(query: str):
    return tavily_client.search(query)

# Create Google search agent
google_search_agent = create_agent(
    model,
    tools=[google_search],
    name="google_search_agent",
    system_prompt=WEB_SEARCH_PROMPT
)  



linkedin_tool = ApifyActorsTool("apimaestro/linkedin-jobs-scraper-api")
linkedin_agent = create_agent(
    model,
    tools = [linkedin_tool],
    system_prompt="search likedIn about the given job",
    name="linkedin job search"
)


In [ ]:
@tool(description="Do a web search based on user queries")
def web_searcher(request: str) -> str:
    result = google_search_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool(description="Resume parsor and answer questions")
def resume_expert(request: str) -> str:
    result = resume_parser_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

@tool(description="Ansewering general quetions")
def friend(request: str) -> str:
    result = general_question_answer_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

@tool(description="Search LinkedIn about given job")
def job_searcher(request: str) -> str:
    result = linkedin_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [ ]:

SUPERVISOR_PROMPT = (
    "You are a Supervisor Agent responsible for coordinating multiple specialized agents.\n\n"

    "Available tools:\n"
    "- `resume_expert`: Use for resume parsing or resume-based questions.\n"
    "- `web_searcher`: Use for current, factual, or real-world information.\n"
    "- `job_searcher`: Use for LinkedIn job searches.\n"
    "- `friend`: Use for general or conversational questions.\n\n"

    "Instructions:\n"
    "- Analyze the user request carefully.\n"
    "- Select the most appropriate tool or tools.\n"
    "- If the task requires multiple steps, call tools sequentially.\n"
    "- Do NOT answer directly; always rely on tools.\n"
)


supervisor_agent = create_agent(
    model,
    tools=[web_searcher, resume_expert,friend,job_searcher],
    system_prompt=SUPERVISOR_PROMPT,
)




In [ ]:
CV_PATH = "Your path"

query = f"""Based on the given resume
{CV_PATH}
find what latest things he must learn and
Also find 10 latest jobs he can search next.
"""

# query = "Hi, how are you, can you help suggest jobs based on my resume"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

I'll help you analyze the resume and find the latest things to learn as well as job opportunities. Let me start by parsing the resume.
Tool Calls:
  resume_expert (call_00_a2gbCCrCgEpAml2EWDXT3Qvg)
 Call ID: call_00_a2gbCCrCgEpAml2EWDXT3Qvg
  Args:
    request: Parse the resume from C://Users//MicroChannel//Downloads//Vegashkar_CV.pdf and provide a comprehensive analysis of the candidate's skills, experience, and current qualifications
================================= Tool Message =================================
Name: resume_expert

Based on the parsed resume, here is a comprehensive analysis of Vegashkar Punyamoorthy's skills, experience, and qualifications:

## **Candidate Overview**
**Name:** Vegashkar Punyamoorthy
**Current Role:** AI/ML Engineer
**Education:** BSc in Electrical and Electronic Engineering, University of Jaffna (GPA: 3.55/4.0)
**Experience Level:** Recent graduate with 1+ years of p

[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> Status: RUNNING, Message: 
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:46.517Z ACTOR: Pulling container image of build RhIlLCLRQUlGIxd0g from registry.
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:46.521Z ACTOR: Creating container.
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:46.571Z ACTOR: Starting container.
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:47.743Z INFO  System info {"apifyVersion":"3.5.0","apifyClientVersion":"2.17.0","crawleeVersion":"3.15.1","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:47.888Z Input: {
[apify.linkedin-jobs-scraper-api runId:dB2RGy4GKFDJKnUQL] -> 2025-12-27T15:13:47.890Z   keywords: 'AI ML Engineer Machine Learning Engineer Data Scientist Python LangChain RAG LLM',
[apify.linkedin-jobs-sc

================================= Tool Message =================================
Name: job_searcher

Based on my search results, I found several AI/ML Engineer, Machine Learning Engineer, and Data Scientist positions with Python, LangChain, RAG, and LLM experience in remote or hybrid positions. Here's a summary of the most relevant opportunities:

## **Top AI/ML Engineer Positions with Your Required Skills:**

### **1. AI Engineer at kadence**
- **Salary**: $150K/yr - $180K/yr
- **Location**: Remote (United States)
- **Key Skills**: LLMs, agentic workflows, tool-calling, RAG, Python, TypeScript
- **Description**: Building agent-driven AI systems with LangGraph, AutoGen, CrewAI experience

### **2. GenAI and Agentic AI Engineer at Arbitration Forums, Inc.**
- **Salary**: $128.6K/yr - $195.9K/yr
- **Location**: Remote (United States)
- **Key Skills**: LLMs, RAG, prompt engineering, Agentic AI frameworks (CrewAI, Azure AI Foundry)

### **3. Senior Engineer II - AI/ML at Fanatics**
- **Sal

# Crewai

In [1]:
from crewai import Crew, Process, Agent